# Setting up SAM

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import io
from PIL import Image
from google.colab import files

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

# Upload an Image

In [ ]:
# 이미지를 업로드해서 사용하는 경우
image_file = files.upload()
image = io.BytesIO(image_file[list(image_file.keys())[0]])
image = np.array(Image.open(image))

# 다운받은 샘플 이미지를 사용하는 경우
# image = cv2.imread('images/dog.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

# Generate Auto Mask

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

### - run `generate` parameter 설정
- 포인트가 얼마나 조밀하게 샘플링 되는지
- 품질이 낮거나 중복된 마스크를 제거하기 위한 임계값 제어
- ...


In [ ]:
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [ ]:
masks2 = mask_generator_2.generate(image)

In [ ]:
len(masks2)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks2)
plt.axis('off')
plt.show()

In [ ]:
#Number of Masks can be found here
print(masks2[0].keys())

# Additional Possibilities: GLCM 
#### (was not used in the research due to the its ambiguity on the outputs from big scenes. It is better suited for small images containing textures )

In [ ]:
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
import imageio
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import io
from PIL import Image
from google.colab import files
import sys

image_file = files.upload()
image = io.BytesIO(image_file[list(image_file.keys())[0]])
image = np.array(Image.open(image))

In [ ]:
# 컬러 이미지를 그레이스케일로 변환합니다.
gray_image = rgb2gray(image)

# 8비트 그레이스케일 이미지로 변환합니다.
gray_image = (gray_image * 255).astype(np.uint8)

# GLCM 계산
glcm = greycomatrix(gray_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

# GLCM 특성 추출
contrast = greycoprops(glcm, 'contrast')[0, 0]
correlation = greycoprops(glcm, 'correlation')[0, 0]
energy = greycoprops(glcm, 'energy')[0, 0]
homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]

plt.figure(figsize=(20,20))
plt.imshow(gray_image)
plt.axis('off')
plt.show()

# print(f'Contrast: {contrast}')
# print(f'Correlation: {correlation}')
# print(f'Energy: {energy}')
# print(f'Homogeneity: {homogeneity}')

In [ ]:
# Feature from GLCM

print(f'Contrast: {contrast}')
print(f'Correlation: {correlation}')
print(f'Energy: {energy}')
print(f'Homogeneity: {homogeneity}')